In [ ]:
import numpy as N
from SWOTRiver import SWOTL2
from SWOTRiver import ReachExtractor
from SWOTRiver import WidthDataBase

%pylab inline

# This is for making changes on the fly

%load_ext autoreload
%autoreload 2

In [ ]:
width_db_dir = '/Volumes/Reservoir/Data/GWD-LR/nAmerica_GRWDL/'
db_file = width_db_dir+'nAmerica_GRWDL.h5'

data_dir = '/Volumes/Reservoir/Data/SWOT/'
#l2_file = (data_dir +
#    'swot_intf_line1_cycle_0001_pass_0277_w0.23_15dB_slp0.1deg.LeftSwath.nc')
l2_file = (data_dir +
'swot_intf_line2-9_cycle_0001_pass_0014_w0.23_15dB_dem0.1slp.RightSwath.nc')
db_dir = '../../data/nAmerica_GRWDL_river_topo/'
shape_file_root = db_dir+'nAmerica_GRWDL_river_topo'

In [ ]:
l2 = SWOTL2(l2_file)

In [ ]:
clip_buffer = 0.02

clip = False
reaches_no_clip = ReachExtractor(shape_file_root, l2,clip=clip,
                             clip_buffer=clip_buffer)

clip = True
reaches_clip = ReachExtractor(shape_file_root, l2,clip=clip,
                             clip_buffer=clip_buffer)

In [ ]:
print reaches_no_clip.reach_idx, reaches_clip.reach_idx
print reaches_no_clip[0].lon.shape, reaches_clip[0].lon.shape

In [ ]:
db = WidthDataBase(db_file)

In [ ]:
reach_index = reaches_no_clip.reach_idx[0]

lon_nc,lat_nc = db.get_lon_lat(reach_index)
print 'no clip length: %d'%len(lon_nc)

lon_c,lat_c,inbbox = db.get_lon_lat(reach_index,
                             bounding_box=l2.bounding_box,
                             clip_buffer=clip_buffer)

print 'clip length: %d'%len(lon_c)
                               
figsize(10,5)
subplot(1,2,1)
plot(lon_nc,lat_nc,'.',alpha=0.1)

subplot(1,2,2)
plot(lon_c,lat_c,'.',alpha=0.1)


In [ ]:
reach_index = reaches_no_clip.reach_idx[0]

x_nc,y_nc = db.get_xy(reach_index,l2.proj)
print 'no clip length: %d'%len(x_nc)

x_c,y_c = db.get_xy(reach_index,l2.proj,
                             bounding_box=l2.bounding_box,
                             clip_buffer=clip_buffer)

print 'clip length: %d'%len(x_c)
                               
figsize(10,5)
subplot(1,2,1)
plot(x_nc,y_nc,'.',alpha=0.1)

subplot(1,2,2)
plot(x_c,y_c,'.',alpha=0.1)


In [ ]:
reach_index = reaches_no_clip.reach_idx[0]

lon_nc,lat_nc,width_nc = db.get_river(reach_index,
                                      columns=['long','lat','width'],
                             asarray=True,transpose=True)
print 'no clip length: %d'%len(lon_nc)

lon_c,lat_c,width_c = db.get_river(reach_index,
                                      columns=['long','lat','width'],
                             asarray=True,transpose=True,
                             bounding_box=l2.bounding_box,
                             clip_buffer=clip_buffer
                             )
print 'clip length: %d'%len(lon_c)

In [ ]:
figsize(10,5)

subplot(1,2,1)
scatter(lon_nc,lat_nc,c=width_nc,edgecolor='none',alpha=0.1)

subplot(1,2,2)
scatter(lon_c,lat_c,c=width_c,edgecolor='none',alpha=0.1)